In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch as t
import pandas as pd
import os
from tqdm import tqdm
from utils import collect_acts
from generate_acts import load_llama
from probes import LRProbe, MMProbe, CCSProbe
import plotly.express as px
import json

In [3]:
model_size = '13B-2'
device = 'cuda:0'

tokenizer, model = load_llama(model_size, device)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


/storage/models/llama-2-13B-GPTQ-4bit


In [9]:
prompt = """\
The Spanish word 'jirafa' means 'giraffe'. This statement is: TRUE
The Spanish word 'escribir' means 'to write'. This statement is: TRUE
The Spanish word 'diccionario' means 'dictionary'. This statement is: TRUE
The Spanish word 'gato' means 'cat'. This statement is: TRUE
The Spanish word 'aire' means 'silver'. This statement is: FALSE"""

statement="The Spanish word 'hola' means 'hello'."
input_ids = tokenizer(prompt + '\n' +  statement + ' This statement is:', return_tensors='pt').input_ids.to(device)
probs = model(input_ids).logits[0,-1,:].softmax(-1)

In [10]:
t_tok = tokenizer.encode('TRUE')[-1]
f_tok = tokenizer.encode('FALSE')[-1]

probs[t_tok], probs[f_tok]

(tensor(0.8439, device='cuda:0', grad_fn=<SelectBackward0>),
 tensor(0.1513, device='cuda:0', grad_fn=<SelectBackward0>))

In [9]:
outputs = model(input_ids, output_hidden_states=True)

In [64]:
outputs.keys()

odict_keys(['logits', 'past_key_values', 'hidden_states'])

In [65]:
len(outputs.past_key_values)

40

In [66]:
len(outputs.past_key_values[0])

2

In [67]:
outputs.past_key_values[0][0].shape

torch.Size([1, 40, 107, 128])

In [68]:
outputs.logits.shape

torch.Size([1, 107, 32000])

In [70]:
len(outputs.hidden_states)

41

In [72]:
outputs.hidden_states[0].shape

torch.Size([1, 107, 5120])

In [128]:
# model.named_parameters()

for name, param in model.named_parameters():
    print(name, param.size())

# for module in model.children():
#     print('hi')
#     print(module)

model.model

model.embed_tokens.weight torch.Size([32000, 5120])
model.layers.0.input_layernorm.weight torch.Size([5120])
model.layers.0.post_attention_layernorm.weight torch.Size([5120])
model.layers.1.input_layernorm.weight torch.Size([5120])
model.layers.1.post_attention_layernorm.weight torch.Size([5120])
model.layers.2.input_layernorm.weight torch.Size([5120])
model.layers.2.post_attention_layernorm.weight torch.Size([5120])
model.layers.3.input_layernorm.weight torch.Size([5120])
model.layers.3.post_attention_layernorm.weight torch.Size([5120])
model.layers.4.input_layernorm.weight torch.Size([5120])
model.layers.4.post_attention_layernorm.weight torch.Size([5120])
model.layers.5.input_layernorm.weight torch.Size([5120])
model.layers.5.post_attention_layernorm.weight torch.Size([5120])
model.layers.6.input_layernorm.weight torch.Size([5120])
model.layers.6.post_attention_layernorm.weight torch.Size([5120])
model.layers.7.input_layernorm.weight torch.Size([5120])
model.layers.7.post_attention_

LlamaModel(
  (embed_tokens): Embedding(32000, 5120, padding_idx=0)
  (layers): ModuleList(
    (0-39): 40 x LlamaDecoderLayer(
      (self_attn): LlamaSdpaAttention(
        (rotary_emb): LlamaRotaryEmbedding()
        (k_proj): QuantLinear()
        (o_proj): QuantLinear()
        (q_proj): QuantLinear()
        (v_proj): QuantLinear()
      )
      (mlp): LlamaMLP(
        (act_fn): SiLU()
        (down_proj): QuantLinear()
        (gate_proj): QuantLinear()
        (up_proj): QuantLinear()
      )
      (input_layernorm): LlamaRMSNorm()
      (post_attention_layernorm): LlamaRMSNorm()
    )
  )
  (norm): LlamaRMSNorm()
)

In [134]:
model.model.layers

ModuleList(
  (0-39): 40 x LlamaDecoderLayer(
    (self_attn): LlamaSdpaAttention(
      (rotary_emb): LlamaRotaryEmbedding()
      (k_proj): QuantLinear()
      (o_proj): QuantLinear()
      (q_proj): QuantLinear()
      (v_proj): QuantLinear()
    )
    (mlp): LlamaMLP(
      (act_fn): SiLU()
      (down_proj): QuantLinear()
      (gate_proj): QuantLinear()
      (up_proj): QuantLinear()
    )
    (input_layernorm): LlamaRMSNorm()
    (post_attention_layernorm): LlamaRMSNorm()
  )
)